# In this notebook, we will use the 2,000 bridged image dataset, RUL dataset, and bridged sensor dataset to predict our final outcome. This process is called model **fusion**. From here, we will determine if a bridge's condition is good, moderate, or bad.

In [46]:
import pandas as pd
import   torch
import joblib
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import os
import torch
import numpy as np
from torchvision import datasets,transforms
from torch.utils.data import DataLoader,random_split
import torch.nn as nn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report



In [2]:
os.chdir("../")

In [3]:
brigded_dataset = pd.read_csv("notebooks/transformed_data/bridged_sensor_dataset.csv")
rul_dataset = pd.read_csv("notebooks/transformed_data/rul_dataset.csv")

# Briged Model Prediction

In [4]:
class LSTMNet(nn.Module):
    def __init__(self, input_size, num_hidden, num_layers):
        super().__init__()

        # LSTM Layer
        self.gru = nn.LSTM(
            input_size=input_size,
            hidden_size=num_hidden,
            num_layers=num_layers,
            batch_first=True  # make input shape (batch, seq, features)
        )

        # Linear layer for output
        self.output = nn.Linear(num_hidden, 1)

        # Sigmoid for binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Run through the GRU layer
        out, hidden = self.gru(x)  # out shape: (batch, seq_len, hidden_size)


        # Pass through linear layer
        out = self.output(out)

        # Apply sigmoid activation for BCELoss
        out = self.sigmoid(out)

        return out, hidden



In [5]:
input_size = 23
num_hidden = 20
num_layers = 10

In [6]:
briged_model = LSTMNet(input_size=23,num_hidden=num_hidden,num_layers=num_layers)

In [7]:
briged_model.load_state_dict(torch.load("notebooks/trained_model/brigded_model.pth"))

/tmp/ipykernel_340523/4243719006.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  briged_model.load_state_dict(torch.load("notebooks/trained_model/brigded_model.pth"))


<All keys matched successfully>

In [8]:
brigded_dataset = brigded_dataset.drop("Unnamed: 0",axis=1)

In [9]:
briged_model.eval()

scaler = StandardScaler()
scaled_x_train = scaler.fit_transform(brigded_dataset)
scaled_x_train = torch.tensor(scaled_x_train).float()

briged_model.eval()

with torch.no_grad():
    y_pred,hidden = briged_model(scaled_x_train)
    briged_model_prediction = (y_pred >= 0.5).float()

In [10]:
briged_model_prediction = briged_model_prediction.squeeze().detach().numpy()

In [11]:
all_model_prediction_dataset = pd.DataFrame(data=briged_model_prediction,columns=["briged_model_prediction"])

In [12]:
all_model_prediction_dataset

,briged_model_prediction
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
2995,0.0
2996,0.0
2997,0.0
2998,0.0


# RUL Model Prediction

In [13]:
rul_dataset = rul_dataset.drop("Unnamed: 0",axis=1)

In [14]:
rul_model = joblib.load("notebooks/trained_model/rul_model.pkl")

In [15]:
rul_model_prediction = rul_model.predict(rul_dataset)

In [16]:
all_model_prediction_dataset["rul_model_prediction"] = rul_model_prediction.round()

In [17]:
all_model_prediction_dataset

,briged_model_prediction,rul_model_prediction
0,1.0,58.0
1,0.0,108.0
2,0.0,69.0
3,0.0,138.0
4,0.0,295.0
...,...,...
2995,0.0,214.0
2996,0.0,287.0
2997,0.0,112.0
2998,0.0,233.0


# Bridge Image Prediction

In [18]:
def get_model_optimizer():
    net = ResNetLike(num_classes=1)
    lossFun = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    return net, optimizer, lossFun

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.downsample:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNetLike(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )

        layers = [BasicBlock(self.in_channels, out_channels, stride, downsample)]
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))  # [B, 64, H/2, W/2]
        x = self.pool(x)                        # [B, 64, H/4, W/4]
        x = self.layer1(x)                      # -> [B, 64, H/4, W/4]
        x = self.layer2(x)                      # -> [B, 128, H/8, W/8]
        x = self.layer3(x)                      # -> [B, 256, H/16, W/16]
        x = self.layer4(x)                      # -> [B, 512, H/32, W/32]
        x = self.global_pool(x)                 # -> [B, 512, 1, 1]
        x = torch.flatten(x, 1)                 # -> [B, 512]
        x = self.fc(x)  # Logits

        return x

In [19]:
net = get_model_optimizer()[0]

In [20]:
net.load_state_dict(torch.load("notebooks/trained_model/brigded_image_model.pth"))

/tmp/ipykernel_340523/2448152017.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("notebooks/trained_model/brigded_image_model.pth"))


<All keys matched successfully>

# LOading in image Dataset

In [21]:
# ===========================
# 1️⃣ Define your transformations
# ===========================
transform = transforms.Compose([
    transforms.Resize((224, 224)),                     # Resize images
    transforms.RandomHorizontalFlip(p=0.5),            # Randomly flip images
    transforms.RandomRotation(degrees=15),             # Random rotation
    transforms.ToTensor(),                             # Convert to tensor [C, H, W] in [0, 1]
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ===========================
# 2️⃣ Safe Dataset Loader
# ===========================
class SafeImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        try:
            return super().__getitem__(index)
        except (FileNotFoundError, OSError):
            # Skip the missing/corrupt image by picking another index
            new_index = (index + 1) % len(self)
            print(f"⚠️ Skipping broken image at index {index}")
            return self.__getitem__(new_index)

# ===========================
# 3️⃣ Load the full dataset safely
# ===========================
dataset = SafeImageFolder(root="data/image_dataset/road_image_dataset", transform=transform)

# ===========================
# 4️⃣ Split dataset into train & test
# ===========================
train_size = int(0.075 * len(dataset))
test_size = len(dataset) - train_size
image_data, _ = random_split(dataset, [train_size, test_size])

# ===========================
# 5️⃣ Create DataLoaders
# ===========================
data_loader = DataLoader(image_data, batch_size=32, shuffle=True)



In [22]:
print(f"✅ Dataset loaded successfully: {len(image_data)} train samples)")

✅ Dataset loaded successfully: 3000 train samples)


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
net.eval()  # set model to evaluation mode
bridged_image_prediction = []

with torch.no_grad():
    for X_val, _ in data_loader:   
        y_pred = net(X_val)        

        preds = (y_pred >= 0.5).float()  
        bridged_image_prediction.extend(preds.tolist())  # add batch results to list

print("✅ Predictions completed:", len(bridged_image_prediction))

✅ Predictions completed: 3000


In [25]:
bridged_image_prediction = np.array(bridged_image_prediction).squeeze()

In [26]:
bridged_image_prediction

array([0., 0., 1., ..., 1., 1., 1.])

In [27]:
all_model_prediction_dataset["bridged_image_prediction"] = bridged_image_prediction

* bride_image_prediction: (0 -> GOOD Bridge) (1 -> Bad Bridge)
* rul_model_prediction: >= 180 GOOD (50 < RUL < 180 → Moderate) (< 50 bad )
* briged_model_prediction: - `0` → Standing  - `1` → Collapsed


In [29]:
all_model_prediction_dataset

,briged_model_prediction,rul_model_prediction,bridged_image_prediction
0,1.0,58.0,0.0
1,0.0,108.0,0.0
2,0.0,69.0,1.0
3,0.0,138.0,0.0
4,0.0,295.0,1.0
...,...,...,...
2995,0.0,214.0,1.0
2996,0.0,287.0,0.0
2997,0.0,112.0,1.0
2998,0.0,233.0,1.0


In [34]:
def create_target(row):
    # Map image prediction
    image_cond = 'Good' if row['bridged_image_prediction'] == 0 else 'Bad'
    
    # Map RUL
    if row['rul_model_prediction'] >= 180:
        rul_cond = 'Good'
    elif 50 < row['rul_model_prediction'] < 180:
        rul_cond = "Moderate"
    else:
        rul_cond = 'Bad'
        
    
    
    # Map bridged prediction
    struct_cond = 'Standing' if row['briged_model_prediction'] == 0 else 'Collapsed'
    
    # Fusion: Prioritize negatives

    if 'Bad' in [image_cond, rul_cond] or struct_cond == 'Collapsed':
        return 0 # Bad
    elif image_cond == 'Good' and rul_cond == 'Good' and struct_cond == 'Standing':
        return 2 # Good
    elif image_cond == 'Good' and rul_cond == "Moderate" and struct_cond == 'Standing':
        return 1 # Moderate


all_model_prediction_dataset['fusion_taget'] = all_model_prediction_dataset.apply(create_target, axis=1)

In [35]:
all_model_prediction_dataset

,briged_model_prediction,rul_model_prediction,bridged_image_prediction,fusion_taget
0,1.0,58.0,0.0,0
1,0.0,108.0,0.0,1
2,0.0,69.0,1.0,0
3,0.0,138.0,0.0,1
4,0.0,295.0,1.0,0
...,...,...,...,...
2995,0.0,214.0,1.0,0
2996,0.0,287.0,0.0,2
2997,0.0,112.0,1.0,0
2998,0.0,233.0,1.0,0


In [36]:
all_model_prediction_dataset["fusion_taget"].value_counts()

fusion_taget
0    1807
1     631
2     562
Name: count, dtype: int64

In [39]:
X,y = all_model_prediction_dataset.drop("fusion_taget",axis=1),all_model_prediction_dataset["fusion_taget"]

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
y_train.value_counts()

fusion_taget
0    1456
1     511
2     433
Name: count, dtype: int64

In [43]:
model = RandomForestClassifier(class_weight="balanced")

In [44]:
model.fit(X_train,y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [45]:
y_pred = model.predict(X_test)

In [47]:
clr = classification_report(y_test,y_pred)

In [48]:
print(clr)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       351
           1       1.00      1.00      1.00       120
           2       1.00      1.00      1.00       129

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



# Good To Go 

In [56]:
model.predict(np.array([[0,60,0]]))

/home/leksman/conda/envs/mycondaenv/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([1])